<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-librariers-and-packages" data-toc-modified-id="Import-librariers-and-packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import librariers and packages</a></span></li><li><span><a href="#Creating-scraping-function-to-get-leaderboard-usernames" data-toc-modified-id="Creating-scraping-function-to-get-leaderboard-usernames-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating scraping function to get leaderboard usernames</a></span></li><li><span><a href="#Using-scraper-to-get-usernames" data-toc-modified-id="Using-scraper-to-get-usernames-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Using scraper to get usernames</a></span></li><li><span><a href="#Cleaning-dataframe" data-toc-modified-id="Cleaning-dataframe-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Cleaning dataframe</a></span></li><li><span><a href="#Saving-dataframe-to-a-csv" data-toc-modified-id="Saving-dataframe-to-a-csv-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Saving dataframe to a csv</a></span></li></ul></div>

# Import librariers and packages

In [1]:
import pandas as pd
import os
import re
import requests
from bs4 import BeautifulSoup
import datetime
from time import sleep 
from selenium import webdriver
 
     
        

# Creating scraping function to get leaderboard usernames

In [17]:
def leaderboard_scraper(soup):
    for leaderboard in soup.find_all('div', {'class': 'allUsersContainer'}):
        for users in leaderboard.find_all('a', {'href': True}):
            user_name = users.attrs['href']
            df_username.loc[len(df_username)] = [user_name]

# Using scraper to get usernames

In [18]:
url = 'https://www.trip.skyscanner.com/leaderboard/region/2000000000399'
df_username = pd.DataFrame(columns=['user_name'])

driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver") 
driver.implicitly_wait(5)
driver.get(url)
sleep(1)
cookies = driver.find_element_by_xpath('//*[@id="cookie-banner-root"]/section/div/div/div/button')
cookies.click()
sleep(5)

for i in range(1, 12): # loop through the leaderboard 12 times to get all 200 usernames
    soup_object = BeautifulSoup(driver.page_source, 'lxml')
    next_button=driver.find_element_by_link_text('Show more')
    next_button.click()
    sleep(5)
leaderboard_scraper(soup_object)
driver.quit()

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.14.0 x86_64)


# Cleaning dataframe

In [ ]:
df_username.head() # view dataframe

In [ ]:
df_username.duplicated().value_counts() # see how many usernames are duplicate

In [ ]:
df_username = df_username.drop_duplicates() # drop duplicate usernames

In [ ]:
df_username.head() # view updated dataframe

In [ ]:
df_username['user_name'].map(lambda x: x.split('/')) # split username

In [ ]:
df_username = df_username[:-1] # ignore last row which is a #

In [ ]:
df_username['id'] = df_username['user_name'].map(lambda x: x.split('/')[4]) # split username from link

In [ ]:
df_username.head() # view df

# Saving dataframe to a csv

In [ ]:
df_username.to_csv('./atx_leaderboards_top200.csv') 